In [3]:
import requests, os, string
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import glob
import numpy as np

In [4]:
chromeOptions = Options()
chromeOptions.add_experimental_option("prefs", {"download.default_directory": r"C:\Users\alexi\OneDrive\Documents\Phone Orders\OrderSheetRawExcel"})

In [5]:
user = ""
user_pass = ""

In [6]:
cpath = r"C:\Users\alexi\Onedrive\chromedriver.exe"
url = "http://www.myrtpos.com/newbdi/index.fwx"
driver = webdriver.Chrome(cpath, options=chromeOptions)
driver.get(url)

In [7]:
driver.find_element_by_id('secUserID').send_keys(user)
driver.find_element_by_id ('secPassword').send_keys(user_pass)
driver.find_element_by_class_name('submit').click()

In [8]:
order_url = 'http://www.myrtpos.com/newbdi/ReorderReport_Calculated_Week2.fwx'
response2 = driver.get(order_url)

In [9]:
market1 = "KANSAS CITY"

In [10]:
select = Select(driver.find_element_by_id('frmMarketID'))
# select by value 
select.select_by_value(market1)

In [11]:
select = Select(driver.find_element_by_name('frmSerialized'))
# select by value 
select.select_by_value('Yes')

In [12]:
driver.find_element_by_id("chkByStore").click()
driver.find_element_by_id("chkAllInv").click()

In [13]:
buttons= driver.find_elements_by_class_name("button-inner") # Selects Both "Generate" and "Raw Excel" Buttons

In [14]:
buttons= driver.find_elements_by_class_name("button-inner")
buttons[0].click() # Generate File

In [15]:
buttons= driver.find_elements_by_class_name("button-inner")
buttons[1].click() # Raw Excel Download

In [17]:
list_of_files = glob.glob(r"C:\Users\alexi\OneDrive\Documents\Phone Orders\OrderSheetRawExcel\*") # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

C:\Users\alexi\OneDrive\Documents\Phone Orders\OrderSheetRawExcel\ReorderReport_5PO117JKE.xls


In [18]:
kc = pd.read_excel(f"{latest_file}")
kc_df = kc[["itmdesc", "orderqty"]]
kc_sums = kc_df.groupby("itmdesc").sum()


kc_sums = kc_sums.replace(0, np.nan)
kc_sums = kc_sums.dropna(how='all', axis=0)

In [22]:
kc.head()

,custno,company,item,itmdesc,manufacturer,qty,onorderqty,orderqty,totalqty,weeksworth,totsold7,totsold14,totsold30,minqty,maxqty
0,CTXKCARM,ARMOUR,'610214658616,ALCATEL 1X EVOLVE,ALCATEL,0,0,0,0,0,0,0,1,0,0
1,CTXKCARM,ARMOUR,'610214661654,ALCATEL 3V,ALCATEL,0,0,0,0,0,0,0,3,0,0
2,CTXKCARM,ARMOUR,'610214657398,ALCATEL 7,ALCATEL,0,0,0,0,0,0,0,0,0,0
3,CTXKCARM,ARMOUR,'610214648259,ALCATEL GO FLIP,ALCATEL,0,0,0,0,0,0,0,0,0,0
4,CTXKCARM,ARMOUR,'610214661470,ALCATEL GO FLIP 3,ALCATEL,0,0,0,0,0,0,0,0,0,0


In [23]:
kc_allocations= []
for i, j in kc_sums.iterrows():
    answer = input(f"How much of {i} were we allocated?")
    kc_allocations.append(answer)

How much of ALCATEL 1X EVOLVE were we allocated?10
How much of ALCATEL 3V were we allocated?5
How much of ALCATEL GO FLIP were we allocated?0
How much of ALCATEL GO FLIP 3 were we allocated?10
How much of APL IPHONE 6S 32G ROS TMUS KIT were we allocated?0
How much of IPHONE 11 (64 GB) BLK were we allocated?3
How much of IPHONE 7 (32GB) BLACK were we allocated?10
How much of IPHONE 7 PLUS (32GB) BLACK were we allocated?0
How much of LG K40 were we allocated?35
How much of LG LMX220 ARISTO 3+ 16G BLK TMUS KIT were we allocated?0
How much of LG Q7+ were we allocated?0
How much of LG STYLO 5 32G WHT were we allocated?50
How much of LG X320 ARISTO 4+ 16G GRY TMUS KIT were we allocated?15
How much of METROSMART HTSPT BLK TMUS KIT were we allocated?0
How much of METROSMART RIDE 600MHZ were we allocated?0
How much of MOT E6 16G BLU TMUS KIT were we allocated?0
How much of MOT G7 PLAY 32G BLU TMUS KIT were we allocated?0
How much of MOT G7 POWER 32G BLU TMUS KIT were we allocated?0
How much of 

In [24]:
# Count of Not 0 Values
types_allocated = 0

for each in kc_allocations:
    if each > "0":
        types_allocated += 1
print(f'{market1} had {types_allocated} types of Devices Allocated')

KANSAS CITY had 10 types of Devices Allocated


In [25]:
kc_sums['allocation'] = kc_allocations

In [26]:
kc_sums = kc_sums.astype({'orderqty':float,'allocation':float})
kc_sums['order_percent'] = kc_sums['orderqty']/kc_sums['allocation']
kc_sums = kc_sums[['order_percent']]
kc_sums

,order_percent
itmdesc,
ALCATEL 1X EVOLVE,10.200000
ALCATEL 3V,46.200000
ALCATEL GO FLIP,inf
ALCATEL GO FLIP 3,1.700000
APL IPHONE 6S 32G ROS TMUS KIT,inf
IPHONE 11 (64 GB) BLK,11.666667
IPHONE 7 (32GB) BLACK,21.200000
IPHONE 7 PLUS (32GB) BLACK,inf
LG K40,6.000000


In [27]:
new_kc_sheet = pd.merge(kc_sums, kc, on='itmdesc', how='left')
new_kc_sheet = new_kc_sheet[['itmdesc', 'company','qty','onorderqty','orderqty','totsold7','totsold14','totsold30','order_percent']]
new_kc_sheet["actual_order"]=new_kc_sheet["orderqty"]/new_kc_sheet["order_percent"]
new_kc_sheet["totalqty"]=new_kc_sheet[['qty','onorderqty','orderqty','actual_order']].sum(axis=1)
new_kc_sheet['weeksworth'] = new_kc_sheet['totalqty']/new_kc_sheet['totsold7']
kc_calculated_sheet=new_kc_sheet[['itmdesc','company','qty','onorderqty','orderqty','actual_order','totalqty','weeksworth','totsold7','totsold14','totsold30','order_percent']]
KC_Orders = kc_calculated_sheet.round({"orderqty":0,"actual_order":0,"weeksworth":2})

KC_Orders

,itmdesc,company,qty,onorderqty,orderqty,actual_order,totalqty,weeksworth,totsold7,totsold14,totsold30,order_percent
0,ALCATEL 1X EVOLVE,ARMOUR,0,0,0,0.0,0.000000,NaN,0,0,1,10.2
1,ALCATEL 1X EVOLVE,BLUE RIDGE,0,0,4,0.0,4.392157,4.39,1,4,4,10.2
2,ALCATEL 1X EVOLVE,EAST 291,3,0,1,0.0,4.098039,4.10,1,3,4,10.2
3,ALCATEL 1X EVOLVE,HIGHWAY 24,1,0,23,2.0,26.254902,4.38,6,10,14,10.2
4,ALCATEL 1X EVOLVE,HOLMES,3,0,0,0.0,3.000000,inf,0,0,1,10.2
5,ALCATEL 1X EVOLVE,NORTH INDY,0,0,36,4.0,39.529412,4.39,9,13,20,10.2
6,ALCATEL 1X EVOLVE,NOLAND,1,5,3,0.0,9.294118,9.29,1,5,17,10.2
7,ALCATEL 1X EVOLVE,PROSPECT,8,0,0,0.0,8.000000,8.00,1,3,7,10.2
8,ALCATEL 1X EVOLVE,STATE,3,0,13,1.0,17.274510,4.32,4,6,11,10.2
9,ALCATEL 1X EVOLVE,TROOST,2,0,10,1.0,12.980392,4.33,3,3,10,10.2


In [130]:
no_sim_kc = kc[~kc.itmdesc.str.contains('SIM')]
no_demo_kc = no_sim_kc[~no_sim_kc.itmdesc.str.contains('DEMO')]
kc_store_percent = no_demo_kc[["company","qty","onorderqty"]]

kc_store_percent = kc_store_percent.astype({'qty':float,'onorderqty':float})
kc_store_percent['total store inv']=kc_store_percent["qty"]+kc_store_percent["onorderqty"]
kc_store_percent = kc_store_percent.drop(columns=["qty", "onorderqty"])
kc_store_percent = kc_store_percent.groupby("company").sum()
combined_kc = pd.merge(KC_Orders, kc_store_percent, on='company', how='left', suffixes=(True, False))

In [131]:
combined_kc["% device of store inv"] = (combined_kc[["onorderqty","qty"]].sum(axis=1)/combined_kc['total store inv'])*100
combined_kc["request"] = combined_kc["orderqty"]-combined_kc["actual_order"]
# combined_kc

In [132]:
KC_Final_Orders= combined_kc[['itmdesc','company','qty','onorderqty','actual_order','request','totalqty','weeksworth','totsold7','totsold14','totsold30','order_percent','total store inv','% device of store inv']]
KC_Final_Orders = KC_Final_Orders.round({"orderqty":0,"actual_order":0,"totalqty":0,"% device of store inv":2})

In [133]:
KC_Final_Orders

,itmdesc,company,qty,onorderqty,actual_order,request,totalqty,weeksworth,totsold7,totsold14,totsold30,order_percent,total store inv,% device of store inv
0,ALCATEL 1X EVOLVE,ARMOUR,0,0,0.0,0.0,0.0,NaN,0,0,1,10.2,0.0,NaN
1,ALCATEL 1X EVOLVE,BLUE RIDGE,0,0,0.0,4.0,4.0,4.39,1,4,4,10.2,123.0,0.00
2,ALCATEL 1X EVOLVE,EAST 291,3,0,0.0,1.0,4.0,4.10,1,3,4,10.2,158.0,1.90
3,ALCATEL 1X EVOLVE,HIGHWAY 24,1,0,2.0,21.0,26.0,4.38,6,10,14,10.2,158.0,0.63
4,ALCATEL 1X EVOLVE,HOLMES,3,0,0.0,0.0,3.0,inf,0,0,1,10.2,149.0,2.01
5,ALCATEL 1X EVOLVE,NORTH INDY,0,0,4.0,32.0,40.0,4.39,9,13,20,10.2,127.0,0.00
6,ALCATEL 1X EVOLVE,NOLAND,1,5,0.0,3.0,9.0,9.29,1,5,17,10.2,222.0,2.70
7,ALCATEL 1X EVOLVE,PROSPECT,8,0,0.0,0.0,8.0,8.00,1,3,7,10.2,151.0,5.30
8,ALCATEL 1X EVOLVE,STATE,3,0,1.0,12.0,17.0,4.32,4,6,11,10.2,200.0,1.50
9,ALCATEL 1X EVOLVE,TROOST,2,0,1.0,9.0,13.0,4.33,3,3,10,10.2,152.0,1.32


In [40]:
# Repeat for Tampa
market2 = "TAMPA"
select = Select(driver.find_element_by_id('frmMarketID'))
# select by value 
select.select_by_value(market2)

In [41]:
buttons= driver.find_elements_by_class_name("button-inner")
buttons[0].click() # Generate File

In [42]:
buttons= driver.find_elements_by_class_name("button-inner")
buttons[1].click() # Raw Excel Download

In [44]:
list_of_files = glob.glob(r"C:\Users\alexi\OneDrive\Documents\Phone Orders\OrderSheetRawExcel\*") # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

C:\Users\alexi\OneDrive\Documents\Phone Orders\OrderSheetRawExcel\ReorderReport_5PO11EXN9.xls


In [45]:
tpa = pd.read_excel(f"{latest_file}")
tpa_df = tpa[["itmdesc", "orderqty"]]
tpa_sums = tpa_df.groupby("itmdesc").sum()


tpa_sums = tpa_sums.replace(0, np.nan)
tpa_sums = tpa_sums.dropna(how='all', axis=0)

In [46]:
tpa_allocations= []
for i, j in tpa_sums.iterrows():
    answer = input(f"How much of {i} were we allocated?")
    tpa_allocations.append(answer)

How much of ALCATEL 1X EVOLVE were we allocated?10
How much of ALCATEL 3V were we allocated?5
How much of ALCATEL GO FLIP 3 were we allocated?10
How much of ALCATEL JOY TAB were we allocated?0
How much of APL IPHONE 6S 32G ROS TMUS KIT were we allocated?0
How much of APL IPHONE XS MAX 64GB GOLD DEMO were we allocated?0
How much of IPHONE 11 (64 GB) BLK were we allocated?0
How much of IPHONE 11 (64 GB) GREEN were we allocated?0
How much of IPHONE 11 (64 GB) PURPLE were we allocated?0
How much of IPHONE 11 (64 GB) RED were we allocated?5
How much of IPHONE 7 (32GB) BLACK were we allocated?0
How much of IPHONE 7 PLUS (32GB) BLACK were we allocated?0
How much of LG K40 were we allocated?0
How much of LG LMX220 ARISTO 3+ 16G BLK TMUS KIT were we allocated?0
How much of LG STYLO 5 32G WHT were we allocated?50
How much of LG X320 ARISTO 4+ 16G GRY TMUS KIT were we allocated?30
How much of METROSMART HTSPT BLK TMUS KIT were we allocated?0
How much of MOT E6 16G BLU TMUS KIT were we allocated?0

In [47]:
# Count of Not 0 Values
tpa_types_allocated = 0

for each in tpa_allocations:
    if each > "0":
        tpa_types_allocated += 1
print(f'{market2} had {tpa_types_allocated} types of Devices Allocated')

TAMPA had 8 types of Devices Allocated


In [48]:
tpa_sums['allocation'] = tpa_allocations

In [49]:
tpa_sums = tpa_sums.astype({'orderqty':float,'allocation':float})
tpa_sums['order_percent'] = tpa_sums['orderqty']/tpa_sums['allocation']
tpa_sums = tpa_sums[['order_percent']]

In [50]:
new_tpa_sheet = pd.merge(tpa_sums, tpa, on='itmdesc', how='left')
new_tpa_sheet = new_tpa_sheet[['itmdesc', 'company','qty','onorderqty','orderqty','totsold7','totsold14','totsold30','order_percent']]
new_tpa_sheet["actual_order"]=new_tpa_sheet["orderqty"]/new_tpa_sheet["order_percent"]
new_tpa_sheet["totalqty"]=new_tpa_sheet[['qty','onorderqty','orderqty','actual_order']].sum(axis=1)
new_tpa_sheet['weeksworth'] = new_tpa_sheet['totalqty']/new_tpa_sheet['totsold7']
tpa_calculated_sheet=new_tpa_sheet[['itmdesc','company','qty','onorderqty','orderqty','actual_order','totalqty','weeksworth','totsold7','totsold14','totsold30','order_percent']]
TPA_Orders = tpa_calculated_sheet.round({"orderqty":0,"actual_order":0,"weeksworth":2})

TPA_Orders

,itmdesc,company,qty,onorderqty,orderqty,actual_order,totalqty,weeksworth,totsold7,totsold14,totsold30,order_percent
0,ALCATEL 1X EVOLVE,CELLTRONIX TAMPA HUB,0,0,0,0.0,0.000000,NaN,0,0,0,2.0
1,ALCATEL 1X EVOLVE,EUCLID,0,0,16,8.0,24.000000,6.00,4,5,13,2.0
2,ALCATEL 1X EVOLVE,KENNEDY,0,0,0,0.0,0.000000,NaN,0,2,6,2.0
3,ALCATEL 1X EVOLVE,LOCKWOOD,8,0,0,0.0,8.000000,inf,0,0,0,2.0
4,ALCATEL 1X EVOLVE,NEPTUNE,0,0,0,0.0,0.000000,NaN,0,0,0,2.0
5,ALCATEL 1X EVOLVE,OLDSMAR,0,0,4,2.0,6.000000,6.00,1,1,2,2.0
6,ALCATEL 1X EVOLVE,PALM HARBOR,0,0,0,0.0,0.000000,NaN,0,0,6,2.0
7,ALCATEL 1X EVOLVE,PINELLAS,0,0,0,0.0,0.000000,NaN,0,1,13,2.0
8,ALCATEL 1X EVOLVE,PORT RICHEY,0,0,0,0.0,0.000000,NaN,0,0,3,2.0
9,ALCATEL 1X EVOLVE,TUTTLE,1,0,0,0.0,1.000000,inf,0,1,2,2.0


In [134]:
no_sim_tpa = tpa[~tpa.itmdesc.str.contains('SIM')]
no_demo_tpa = no_sim_tpa[~no_sim_tpa.itmdesc.str.contains('DEMO')]
tpa_store_percent = no_demo_tpa[["company","qty","onorderqty"]]

tpa_store_percent = tpa_store_percent.astype({'qty':float,'onorderqty':float})
tpa_store_percent['total store inv']=tpa_store_percent["qty"]+tpa_store_percent["onorderqty"]
tpa_store_percent = tpa_store_percent.drop(columns=["qty", "onorderqty"])
tpa_store_percent = tpa_store_percent.groupby("company").sum()
combined_tpa = pd.merge(TPA_Orders, tpa_store_percent, on='company', how='left', suffixes=(True, False))

In [135]:
combined_tpa["% device of store inv"] = (combined_tpa[["onorderqty","qty"]].sum(axis=1)/combined_tpa['total store inv'])*100
combined_tpa["request"] = combined_tpa["orderqty"]-combined_tpa["actual_order"]

In [136]:
TPA_Final_Orders= combined_tpa[['itmdesc','company','qty','onorderqty','actual_order','request','totalqty','weeksworth','totsold7','totsold14','totsold30','order_percent','total store inv','% device of store inv']]
TPA_Final_Orders = TPA_Final_Orders.round({"orderqty":0,"actual_order":0,"totalqty":0,"% device of store inv":2})

In [137]:
TPA_Final_Orders

,itmdesc,company,qty,onorderqty,actual_order,request,totalqty,weeksworth,totsold7,totsold14,totsold30,order_percent,total store inv,% device of store inv
0,ALCATEL 1X EVOLVE,CELLTRONIX TAMPA HUB,0,0,0.0,0.0,0.0,NaN,0,0,0,2.0,0.0,NaN
1,ALCATEL 1X EVOLVE,EUCLID,0,0,8.0,8.0,24.0,6.00,4,5,13,2.0,280.0,0.00
2,ALCATEL 1X EVOLVE,KENNEDY,0,0,0.0,0.0,0.0,NaN,0,2,6,2.0,240.0,0.00
3,ALCATEL 1X EVOLVE,LOCKWOOD,8,0,0.0,0.0,8.0,inf,0,0,0,2.0,197.0,4.06
4,ALCATEL 1X EVOLVE,NEPTUNE,0,0,0.0,0.0,0.0,NaN,0,0,0,2.0,0.0,NaN
5,ALCATEL 1X EVOLVE,OLDSMAR,0,0,2.0,2.0,6.0,6.00,1,1,2,2.0,263.0,0.00
6,ALCATEL 1X EVOLVE,PALM HARBOR,0,0,0.0,0.0,0.0,NaN,0,0,6,2.0,207.0,0.00
7,ALCATEL 1X EVOLVE,PINELLAS,0,0,0.0,0.0,0.0,NaN,0,1,13,2.0,339.0,0.00
8,ALCATEL 1X EVOLVE,PORT RICHEY,0,0,0.0,0.0,0.0,NaN,0,0,3,2.0,231.0,0.00
9,ALCATEL 1X EVOLVE,TUTTLE,1,0,0.0,0.0,1.0,inf,0,1,2,2.0,174.0,0.57


In [ ]:
# Perform the Same Operations for Orlando

In [80]:
market3 = "ORLANDO"
select = Select(driver.find_element_by_id('frmMarketID'))
# select by value 
select.select_by_value(market3)

In [81]:
buttons= driver.find_elements_by_class_name("button-inner")
buttons[0].click() # Generate File

In [82]:
buttons= driver.find_elements_by_class_name("button-inner")
buttons[1].click() # Raw Excel Download

In [83]:
driver.quit() # Close Chrome Driver 

In [84]:
list_of_files = glob.glob(r"C:\Users\alexi\OneDrive\Documents\Phone Orders\OrderSheetRawExcel\*") # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

C:\Users\alexi\OneDrive\Documents\Phone Orders\OrderSheetRawExcel\ReorderReport_5PO11YTZL.xls


In [85]:
orl = pd.read_excel(f"{latest_file}")
orl_df = orl[["itmdesc", "orderqty"]]
orl_sums = orl_df.groupby("itmdesc").sum()


orl_sums = orl_sums.replace(0, np.nan)
orl_sums = orl_sums.dropna(how='all', axis=0)

In [86]:
orl_allocations= []
for i, j in orl_sums.iterrows():
    answer = input(f"How much of {i} were we allocated?")
    orl_allocations.append(answer)

How much of ALCATEL 1X EVOLVE were we allocated?10
How much of ALCATEL 3V were we allocated?5
How much of ALCATEL GO FLIP 3 were we allocated?0
How much of ALCATEL JOY TAB were we allocated?0
How much of APL IPHONE 6S 32G ROS TMUS KIT were we allocated?0
How much of IPHONE 11 (64 GB) BLK were we allocated?0
How much of IPHONE 11 (64 GB) RED were we allocated?0
How much of IPHONE 7 (32GB) BLACK were we allocated?0
How much of IPHONE 7 PLUS (32GB) BLACK were we allocated?0
How much of LG K40 were we allocated?35
How much of LG LMX220 ARISTO 3+ 16G BLK TMUS KIT were we allocated?25
How much of LG Q7+ were we allocated?10
How much of LG STYLO 5 32G WHT were we allocated?0
How much of LG X320 ARISTO 4+ 16G GRY TMUS KIT were we allocated?0
How much of MOT E6 16G BLU TMUS KIT were we allocated?0
How much of MOT G7 PLAY 32G BLU TMUS KIT were we allocated?0
How much of MOT G7 POWER 32G BLU TMUS KIT were we allocated?0
How much of MOTOROLA E5 PLAY were we allocated?10
How much of SAM A205U A20 3

In [87]:
# Count of Not 0 Values
orl_types_allocated = 0

for each in orl_allocations:
    if each > "0":
        orl_types_allocated += 1
print(f'{market3} had {orl_types_allocated} types of Devices Allocated')

ORLANDO had 10 types of Devices Allocated


In [88]:
orl_sums['allocation'] = orl_allocations

In [89]:
orl_sums = orl_sums.astype({'orderqty':float,'allocation':float})
orl_sums['order_percent'] = orl_sums['orderqty']/orl_sums['allocation']
orl_sums = orl_sums[['order_percent']]

In [91]:
new_orl_sheet = pd.merge(orl_sums, orl, on='itmdesc', how='left')
new_orl_sheet = new_orl_sheet[['itmdesc', 'company','qty','onorderqty','orderqty','totsold7','totsold14','totsold30','order_percent']]
new_orl_sheet["actual_order"]=new_orl_sheet["orderqty"]/new_orl_sheet["order_percent"]
new_orl_sheet["totalqty"]=new_orl_sheet[['qty','onorderqty','orderqty','actual_order']].sum(axis=1)
new_orl_sheet['weeksworth'] = new_orl_sheet['totalqty']/new_orl_sheet['totsold7']
orl_calculated_sheet=new_orl_sheet[['itmdesc','company','qty','onorderqty','orderqty','actual_order','totalqty','weeksworth','totsold7','totsold14','totsold30','order_percent']]
ORL_Orders = orl_calculated_sheet.round({"orderqty":0,"actual_order":0,"weeksworth":2})

ORL_Orders.head()

,itmdesc,company,qty,onorderqty,orderqty,actual_order,totalqty,weeksworth,totsold7,totsold14,totsold30,order_percent
0,ALCATEL 1X EVOLVE,APOPKA,1,0,0,0.0,1.000000,inf,0,0,6,5.5
1,ALCATEL 1X EVOLVE,BUMBY,1,0,7,1.0,9.272727,4.64,2,4,5,5.5
2,ALCATEL 1X EVOLVE,CLARCONA,1,0,0,0.0,1.000000,inf,0,0,5,5.5
3,ALCATEL 1X EVOLVE,FOREST,0,0,16,3.0,18.909091,4.73,4,5,6,5.5
4,ALCATEL 1X EVOLVE,GOLDENROD,0,0,0,0.0,0.000000,NaN,0,6,9,5.5


In [138]:
no_sim_orl = orl[~orl.itmdesc.str.contains('SIM')]
no_demo_orl = no_sim_orl[~no_sim_orl.itmdesc.str.contains('DEMO')]
orl_store_percent = no_demo_orl[["company","qty","onorderqty"]]

orl_store_percent = orl_store_percent.astype({'qty':float,'onorderqty':float})
orl_store_percent['total store inv']=orl_store_percent["qty"]+orl_store_percent["onorderqty"]
orl_store_percent = orl_store_percent.drop(columns=["qty", "onorderqty"])
orl_store_percent = orl_store_percent.groupby("company").sum()
combined_orl = pd.merge(ORL_Orders, orl_store_percent, on='company', how='left', suffixes=(True, False))
# combined_orl

In [139]:
combined_orl["% device of store inv"] = (combined_orl[["onorderqty","qty"]].sum(axis=1)/combined_orl['total store inv'])*100
combined_orl["request"] = combined_orl["orderqty"]-combined_orl["actual_order"]

In [140]:
ORL_Final_Orders= combined_orl[['itmdesc','company','qty','onorderqty','actual_order','request','totalqty','weeksworth','totsold7','totsold14','totsold30','order_percent','total store inv','% device of store inv']]
ORL_Final_Orders = ORL_Final_Orders.round({"orderqty":0,"actual_order":0,"totalqty":0,"% device of store inv":2})

In [141]:
ORL_Final_Orders

,itmdesc,company,qty,onorderqty,actual_order,request,totalqty,weeksworth,totsold7,totsold14,totsold30,order_percent,total store inv,% device of store inv
0,ALCATEL 1X EVOLVE,APOPKA,1,0,0.0,0.0,1.0,inf,0,0,6,5.5,163.0,0.61
1,ALCATEL 1X EVOLVE,BUMBY,1,0,1.0,6.0,9.0,4.64,2,4,5,5.5,135.0,0.74
2,ALCATEL 1X EVOLVE,CLARCONA,1,0,0.0,0.0,1.0,inf,0,0,5,5.5,210.0,0.48
3,ALCATEL 1X EVOLVE,FOREST,0,0,3.0,13.0,19.0,4.73,4,5,6,5.5,167.0,0.00
4,ALCATEL 1X EVOLVE,GOLDENROD,0,0,0.0,0.0,0.0,NaN,0,6,9,5.5,181.0,0.00
5,ALCATEL 1X EVOLVE,HIAWASSEE,0,0,1.0,7.0,9.0,4.73,2,2,9,5.5,274.0,0.00
6,ALCATEL 1X EVOLVE,ORANGE,0,0,0.0,0.0,0.0,NaN,0,6,13,5.5,180.0,0.00
7,ALCATEL 1X EVOLVE,ROSEMONT,0,3,1.0,3.0,8.0,7.73,1,1,11,5.5,204.0,1.47
8,ALCATEL 1X EVOLVE,SEMORAN,0,0,1.0,3.0,5.0,4.73,1,3,15,5.5,150.0,0.00
9,ALCATEL 1X EVOLVE,UNDERHILL,0,0,3.0,13.0,19.0,4.73,4,5,13,5.5,256.0,0.00


In [142]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('0304 All Markets_test3.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
ORL_Final_Orders.to_excel(writer, sheet_name='Orlando')
TPA_Final_Orders.to_excel(writer, sheet_name='Tampa')
KC_Final_Orders.to_excel(writer, sheet_name='KC')

In [143]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()